In [1]:
import numpy as np

In [2]:
def distPoint2EpipolarLine(F, p1, p2):
    """ Compute the point-to-epipolar-line distance

       Input:
       - F np.ndarray(3,3): Fundamental matrix
       - p1 np.ndarray(3,N): homogeneous coords of the observed points in image 1
       - p2 np.ndarray(3,N): homogeneous coords of the observed points in image 2

       Output:
       - cost: sum of squared distance from points to epipolar lines
               normalized by the number of point coordinates
    """

    N = p1.shape[1]

    homog_points = np.c_[p1, p2]
    epi_lines = np.c_[F.T @ p2, F @ p1]

    denom = epi_lines[0,:]**2 + epi_lines[1,:]**2
    cost = np.sqrt( np.sum( np.sum( epi_lines * homog_points, axis = 0)**2 / denom) / N)

    return cost

In [86]:
def fundamentalEightPoint(p1, p2):
    """ The 8-point algorithm for the estimation of the fundamental matrix F

     The eight-point algorithm for the fundamental matrix with a posteriori
     enforcement of the singularity constraint (det(F)=0).
     Does not include data normalization.

     Reference: "Multiple View Geometry" (Hartley & Zisserman 2000), Sect. 10.1 page 262.

     Input: point correspondences
      - p1 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 1
      - p2 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 2

     Output:
      - F np.ndarray(3,3) : fundamental matrix
    """

    num_P = p1.shape[1]
    A = np.zeros((num_P, 9))
    for i in range(num_P):
        A[i, :] = np.kron(p1[:, i], p2[:, i]).T
    U, sigma, VT = np.linalg.svd(A, full_matrices=True)
    F = VT.T[:, -1].reshape(3, 3)
    U_F, sigma_F, VT_F = np.linalg.svd(F, full_matrices=True)
    sigma_F[2] = 0
    F_mod = U_F @ np.diag(sigma_F) @ VT_F
    
    return F_mod

## data input and initialize

In [4]:
import os
dirname = os.path.dirname('/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 6/python/code/')

In [5]:
# Number of 3D points to test
N = 40

# Random homogeneous coordinates of 3-D points
X = np.loadtxt(dirname+"/matlab_X.csv", delimiter = ",")

In [6]:
# Simulated scene with error-free correspondances
X[2, :] = X[2, :] * 5 + 10
X[3, :] = 1

In [7]:
# points without noise
P1 = np.array([ [500,   0,      320,    0],
                [0,     500,    240,    0],
                [0,     0,      1,      0]])

P2 = np.array([ [500,   0,      320,    -100],
                [0,     500,    240,    0],
                [0,     0,      1,      0]])

# Image (i.e. projected points)
x1 = P1 @ X
x2 = P2 @ X

In [92]:
# points with noise
sigma = 1e-1
#  noisy_x1 = x1 + sigma * np.random.randn(*x1.shape)
#  noisy_x2 = x2 + sigma * np.random.randn(*x2.shape)

# If you want to get the same results as matlab users, uncomment those two lines
noisy_x1 = np.loadtxt(dirname + "/matlab_noisy_x1.csv", delimiter = ",")
noisy_x2 = np.loadtxt(dirname + "/matlab_noisy_x2.csv", delimiter = ",")

## Test

In [9]:
p1 = x1
p2 = x2

In [10]:
num_P = p1.shape[1]

In [11]:
x1.shape

(3, 40)

array([3080.31674875, 2379.4294425 ,    9.77570825])

In [39]:
pts = x1

In [40]:
pts_E = pts/pts[2,:]

In [41]:
miu = np.mean(pts_E[:2,:], axis = 1)

In [42]:
miu

array([305.39739015, 226.92227626])

In [51]:
pts_dis = pts_E[:2,:].T - miu

In [54]:
A = 3*np.eye(3)

In [60]:
A[1,0] = 6

In [63]:
A[:,:2]**2

array([[ 9.,  0.],
       [36., 36.],
       [ 0.,  0.]])

In [66]:
sigma = np.sqrt(np.mean(np.sum(pts_dis**2, axis = 1)))

128.64400592486217

In [74]:
s = np.sqrt(2) / sigma

In [76]:
    T = np.array([
        [s, 0, -s * miu[0]],
        [0, s, -s * miu[1]],
        [0, 0, 1]])

In [78]:
pts_tilde = T @ pts_E

## Function

In [96]:
import numpy as np

def normalise2DPts(pts):
    """  normalises 2D homogeneous points

     Function translates and normalises a set of 2D homogeneous points
     so that their centroid is at the origin and their mean distance from
     the origin is sqrt(2).

     Usage:   [pts_tilde, T] = normalise2dpts(pts)

     Argument:
       pts -  3xN array of 2D homogeneous coordinates

     Returns:
       pts_tilde -  3xN array of transformed 2D homogeneous coordinates.
       T         -  The 3x3 transformation matrix, pts_tilde = T*pts
    """
    
    pts_E = pts/pts[2:,]
    miu = np.mean(pts_E[:2,:], axis = 1)
    
    pts_dis = pts_E[:2,:].T - miu
    sigma = np.sqrt(np.mean(np.sum(pts_dis**2, axis = 1)))
    s = np.sqrt(2) / sigma
    T = np.array([
    [s, 0, -s * miu[0]],
    [0, s, -s * miu[1]],
    [0, 0, 1]])
    pts_tilde = T @ pts_E
    return pts_tilde, T


In [80]:
import numpy as np


def fundamentalEightPointNormalized(p1, p2):
    """ Normalized Version of the 8 Point algorith
     Input: point correspondences
      - p1 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 1
      - p2 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 2

     Output:
      - F np.ndarray(3,3) : fundamental matrix
    """
    p1_n, T1 = normalise2DPts(p1)
    p2_n, T2 = normalise2DPts(p2)
    
    F_ = fundamentalEightPoint(p1_n, p2_n)
    
    F = T2.T @ F_ @T1
    
    return F


## solution

In [89]:
def normalise2DPts_solution(pts):
    """  normalises 2D homogeneous points

     Function translates and normalises a set of 2D homogeneous points
     so that their centroid is at the origin and their mean distance from
     the origin is sqrt(2).

     Usage:   [pts_tilde, T] = normalise2dpts(pts)

     Argument:
       pts -  3xN array of 2D homogeneous coordinates

     Returns:
       pts_tilde -  3xN array of transformed 2D homogeneous coordinates.
       T         -  The 3x3 transformation matrix, pts_tilde = T*pts
    """


    N = pts.shape[1]

    # Convert homogeneous coordinates to Euclidean coordinates (pixels)
    pts_ = pts/pts[2,:]

    # Centroid (Euclidean coordinates)
    mu = np.mean(pts_[:2,:], axis = 1)

    # Average distance or root mean squared distance of centered points
    # It does not matter too much which criterion to use. Both improve the
    # numerical conditioning of the Fundamental matrix estimation problem.
    pts_centered = (pts_[:2,:].T - mu).T

    # Option 1: RMS distance
    sigma = np.sqrt( np.mean( np.sum(pts_centered**2, axis = 0) ) )
    # axis = 1？
    # Option 2: average distance
    # sigma = mean( sqrt(sum(pts_centered.^2)) );

    s = np.sqrt(2) / sigma
    T = np.array([
        [s, 0, -s * mu[0]],
        [0, s, -s * mu[1]],
        [0, 0, 1]])

    pts_tilde = T @ pts_

    return pts_tilde, T


In [90]:
def fundamentalEightPointNormalized_sol(p1, p2):
    """ Normalized Version of the 8 Point algorith
     Input: point correspondences
      - p1 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 1
      - p2 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 2

     Output:
      - F np.ndarray(3,3) : fundamental matrix
    """

    # Normalize each set of points so that the origin
    # is at centroid and mean distance from origin is sqrt(2).
    p1_tilde, T1 = normalise2DPts_solution(p1)
    p2_tilde, T2 = normalise2DPts_solution(p2)

    # Linear solution
    F = fundamentalEightPoint(p1_tilde, p2_tilde)

    # Undo the normalization
    F = T2.T @ F @ T1

    return F

## Test code

In [93]:
# Test with noise
#  F  = fundamentalEightPoint(noisy_x1, noisy_x2) # This gives bad results!
F  = fundamentalEightPointNormalized(noisy_x1, noisy_x2) # This gives good results!

cost_algebraic = np.linalg.norm( np.sum(noisy_x2 * (F @ noisy_x1), axis=0) ) / np.sqrt(N)
cost_dist_epi_line = distPoint2EpipolarLine(F, noisy_x1, noisy_x2)

print("")
print('Noisy correspondences with normalized 8 Point Algorithm')
print('Algebraic error: %f' % cost_algebraic)
print('Geometric error: %f px' % cost_dist_epi_line)


Noisy correspondences with normalized 8 Point Algorithm
Algebraic error: 4.849496
Geometric error: 63.578559 px


In [94]:
# Test with noise
#  F  = fundamentalEightPoint(noisy_x1, noisy_x2) # This gives bad results!
F  = fundamentalEightPointNormalized_sol(noisy_x1, noisy_x2) # This gives good results!

cost_algebraic = np.linalg.norm( np.sum(noisy_x2 * (F @ noisy_x1), axis=0) ) / np.sqrt(N)
cost_dist_epi_line = distPoint2EpipolarLine(F, noisy_x1, noisy_x2)

print("")
print('Noisy correspondences with normalized solution 8 Point Algorithm')
print('Algebraic error: %f' % cost_algebraic)
print('Geometric error: %f px' % cost_dist_epi_line)


Noisy correspondences with normalized solution 8 Point Algorithm
Algebraic error: 6.804810
Geometric error: 120.160463 px


In [95]:
# Test with noise
F  = fundamentalEightPoint(noisy_x1, noisy_x2) # This gives bad results!

cost_algebraic = np.linalg.norm( np.sum(noisy_x2 * (F @ noisy_x1), axis=0) ) / np.sqrt(N)
cost_dist_epi_line = distPoint2EpipolarLine(F, noisy_x1, noisy_x2)

print("")
print('Noisy correspondences with 8 Point Algorithm')
print('Algebraic error: %f' % cost_algebraic)
print('Geometric error: %f px' % cost_dist_epi_line)



Noisy correspondences with 8 Point Algorithm
Algebraic error: 8.776260
Geometric error: 320.780422 px
